In [1]:
import psycopg2
import pandas as pd
from datetime import datetime, date
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import numpy as np
%matplotlib inline

In [2]:
pg_connection = psycopg2.connect(
    dbname="postgres",
    user="apparao",
    password="!bern@HARD23",
    host="thedb.postgres.database.azure.com",
    port="5432"
)

pg_cursor = pg_connection.cursor()

In [3]:
query = 'select * from pres_election.countypres'

In [4]:
pg_connection = psycopg2.connect(
    dbname="postgres",
    user="apparao",
    password="!bern@HARD23",
    host="thedb.postgres.database.azure.com",
    port="5432"
)

pg_cursor = pg_connection.cursor()
pg_cursor.execute(query)
rows = pg_cursor.fetchall()
county_pres_columns = [col[0] for col  in pg_cursor.description]

In [5]:
county_pres_df = pd.DataFrame(rows, columns=county_pres_columns)

In [6]:
import duckdb
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [7]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [15]:
%%sql

SELECT 
    a.year,
    a.state,
    a.county_name,
    CASE WHEN len(county_fips) == 4 THEN '0' || county_fips ELSE county_fips END as FIPS,
    party, 
    candidatevotes,
    totalvotes,
    candidatevotes / totalvotes as candidates_vote_share,
    CASE year 
        WHEN 2000 THEN 0
        WHEN 2004 THEN
            CASE WHEN party = 'REPUBLICAN' THEN 1 ELSE 0 END 
        WHEN 2008 THEN 0
        WHEN 2012 THEN 
            CASE WHEN party = 'DEMOCRAT' THEN 1 ELSE 0 END
        WHEN 2016 THEN 0 
        WHEN 2020 THEN
            CASE WHEN party = 'REPUBLICAN' THEN 1 ELSE 0 END 
    END as INCUMBENT
FROM county_pres_df a 

,year,state,county_name,FIPS,party,candidatevotes,totalvotes,candidates_vote_share,INCUMBENT
0,2000,ALABAMA,AUTAUGA,01001,DEMOCRAT,4942,17208,0.287192,0
1,2000,ALABAMA,AUTAUGA,01001,REPUBLICAN,11993,17208,0.696943,0
2,2000,ALABAMA,AUTAUGA,01001,GREEN,160,17208,0.009298,0
3,2000,ALABAMA,AUTAUGA,01001,OTHER,113,17208,0.006567,0
4,2000,ALABAMA,BALDWIN,01003,DEMOCRAT,13997,56480,0.247822,0
...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WASHAKIE,56043,REPUBLICAN,3245,4032,0.804812,1
72613,2020,WYOMING,WESTON,56045,DEMOCRAT,360,3560,0.101124,0
72614,2020,WYOMING,WESTON,56045,LIBERTARIAN,46,3560,0.012921,0
72615,2020,WYOMING,WESTON,56045,OTHER,47,3560,0.013202,0
